In [1]:
import statsapi
from datetime import date

today = date.today().isoformat()

# Fetch American League standings for today (leagueId=103)
standings_data = statsapi.get("standings", {"leagueId": 103, "season": 2025, "date": today})

print("Type:", type(standings_data))
print("Top-level keys:", list(standings_data.keys()))


Type: <class 'dict'>
Top-level keys: ['copyright', 'records']


In [2]:
records = standings_data.get("records", [])
print("Number of records (divisions):", len(records))

if records:
    first_record = records[0]
    print("Record keys:", list(first_record.keys()))

    team_records = first_record.get("teamRecords", [])
    print("Number of team records:", len(team_records))

    if team_records:
        print("Team record keys:", list(team_records[0].keys()))
        print("Nested 'team' keys:", list(team_records[0].get("team", {}).keys()))


Number of records (divisions): 3
Record keys: ['standingsType', 'league', 'division', 'sport', 'lastUpdated', 'teamRecords']
Number of team records: 5
Team record keys: ['team', 'season', 'streak', 'divisionRank', 'leagueRank', 'sportRank', 'gamesPlayed', 'gamesBack', 'wildCardGamesBack', 'leagueGamesBack', 'springLeagueGamesBack', 'sportGamesBack', 'divisionGamesBack', 'conferenceGamesBack', 'leagueRecord', 'lastUpdated', 'records', 'runsAllowed', 'runsScored', 'divisionChamp', 'divisionLeader', 'hasWildcard', 'clinched', 'eliminationNumber', 'eliminationNumberSport', 'eliminationNumberLeague', 'eliminationNumberDivision', 'eliminationNumberConference', 'wildCardEliminationNumber', 'magicNumber', 'wins', 'losses', 'runDifferential', 'winningPercentage']
Nested 'team' keys: ['id', 'name', 'link']


In [3]:
import requests
import pandas as pd

clean_rows = []

# Loop through each division record
for rec in records:
    league_id = (rec.get("league") or {}).get("id")
    division_id = (rec.get("division") or {}).get("id")

    for tr in rec.get("teamRecords", []):
        team = tr.get("team", {}) or {}

        row = {
            "season": 2025,
            "date": today,
            "league_id": league_id,
            "division_id": division_id,
            "team_id": team.get("id"),
            "team_name": team.get("name"),
            "wins": tr.get("wins"),
            "losses": tr.get("losses"),
            "pct": tr.get("winningPercentage"),
            "gamesBack": tr.get("gamesBack"),
            "wildCardGamesBack": tr.get("wildCardGamesBack"),
            "divisionLeader": tr.get("divisionLeader"),
        }
        clean_rows.append(row)

df_standings = pd.DataFrame(clean_rows)

# --- League + Division lookups (raw requests) ---
leagues = requests.get("https://statsapi.mlb.com/api/v1/leagues").json()["leagues"]
df_leagues = pd.DataFrame([{"league_id": l["id"], "league": l["name"]} for l in leagues])

divs = requests.get("https://statsapi.mlb.com/api/v1/divisions").json()["divisions"]
df_divs = pd.DataFrame([{"division_id": d["id"], "division": d["name"]} for d in divs])

# Merge lookups
df_standings = df_standings.merge(df_leagues, on="league_id", how="left")
df_standings = df_standings.merge(df_divs, on="division_id", how="left")

print("Shape:", df_standings.shape)
df_standings.head(10)


Shape: (15, 14)


,season,date,league_id,division_id,team_id,team_name,wins,losses,pct,gamesBack,wildCardGamesBack,divisionLeader,league,division
0,2025,2025-09-08,103,201,141,Toronto Blue Jays,82,61,.573,-,-,True,American League,American League East
1,2025,2025-09-08,103,201,147,New York Yankees,80,63,.559,2.0,+5.0,False,American League,American League East
2,2025,2025-09-08,103,201,111,Boston Red Sox,79,65,.549,3.5,+3.5,False,American League,American League East
3,2025,2025-09-08,103,201,139,Tampa Bay Rays,71,72,.497,11.0,4.0,False,American League,American League East
4,2025,2025-09-08,103,201,110,Baltimore Orioles,66,77,.462,16.0,9.0,False,American League,American League East
5,2025,2025-09-08,103,202,116,Detroit Tigers,82,62,.569,-,-,True,American League,American League Central
6,2025,2025-09-08,103,202,118,Kansas City Royals,73,70,.510,8.5,2.0,False,American League,American League Central
7,2025,2025-09-08,103,202,114,Cleveland Guardians,72,70,.507,9.0,2.5,False,American League,American League Central
8,2025,2025-09-08,103,202,142,Minnesota Twins,63,80,.441,18.5,12.0,False,American League,American League Central
9,2025,2025-09-08,103,202,145,Chicago White Sox,55,89,.382,27.0,20.5,False,American League,American League Central


In [4]:
import sqlite3

# Point to the root db file (outside MLB Api Endpoints)
db_path = r"C:\Users\sneal\Dev\mlb-data-pipeline\mlb_data.db"

conn = sqlite3.connect(db_path)

df_standings.to_sql("standings_silver", conn, if_exists="replace", index=False) # in Standings notebook

conn.close()

print("✅ teams_silver written to root mlb_data.db")


✅ teams_silver written to root mlb_data.db
